In [37]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import pickle
from keras.layers import Input, Dense, Lambda, Flatten, Reshape, Layer
from keras.layers import Conv2D, Conv2DTranspose
from keras.models import Model,Sequential
from keras import backend as K
from keras import metrics

In [38]:
img_rows, img_cols, img_chns = 28, 28, 1
filters = 64
num_conv = 3
batch_size = 100
latent_dim = 8
intermediate_dim = 128
epsilon_std = 1.0
epochs = 200
images_count_training = 4
images_count_test = 1
TOTAL_IMAGES = 5
original_img_size = (28,28,1)
out_shape = (batch_size, int(img_rows / 2), int(img_cols / 2), filters)

In [39]:
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
                              mean=0., stddev=epsilon_std)
    return z_mean + K.exp(z_log_var) * epsilon

In [40]:
def Encoder():
    x = Input(shape=original_img_size)
    conv_1 = Conv2D(img_chns,
                    kernel_size=(2, 2),
                    padding='same', activation='relu', name = 'conv_1')(x)
    conv_2 = Conv2D(filters,
                    kernel_size=(2, 2),
                    padding='same', activation='relu',
                    strides=(2, 2),name = 'conv_2')(conv_1)
    conv_3 = Conv2D(filters,
                    kernel_size=num_conv,
                    padding='same', activation='relu',
                    strides=1, name = 'conv_3')(conv_2)
    conv_4 = Conv2D(filters,
                    kernel_size=num_conv,
                    padding='same', activation='relu',
                    strides=1, name = 'conv_4')(conv_3)
    flat = Flatten(name ='flattened_layer')(conv_4)
    hidden = Dense(intermediate_dim, activation='relu', name = 'hidden_layer')(flat)

    z_mean = Dense(latent_dim, name = 'mean_vector')(hidden)
    z_log_var = Dense(latent_dim, name = 'log_var')(hidden)
    
    #funzione che serve per estrarre un elemento dal latent space utilizzando z_mean and z_log_var
    #come input
    z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
    print(z.shape)
    encoder = Model(x, [z_mean, z_log_var, z])
    
    return encoder


In [41]:
def Decoder(output_shape):
    model = Sequential()
    model.add(Dense(intermediate_dim, activation='relu',input_shape = (None,8)))
    model.add(Dense(int(filters * img_rows / 2 * img_cols / 2), activation='relu'))
    model.add(Reshape(output_shape[1:]))
    model.add(Conv2DTranspose(filters, kernel_size=num_conv, padding='same', strides=1, activation='relu'))
    model.add(Conv2DTranspose(filters, kernel_size=num_conv, padding='same', strides=1, activation='relu'))
    model.add(Conv2DTranspose(filters, kernel_size=(3, 3), strides=(2, 2), padding='valid', activation='relu'))
    model.add(Conv2D(img_chns,kernel_size=2, padding='valid',activation='sigmoid'))
    return model